# NLP Project

For this project, we will build a spam filter. We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

data = spark.read.csv("/FileStore/tables/SMSSpamCollection",inferSchema=True,sep='\t')
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

In [3]:
# Clean and Prepare the Data
from pyspark.sql.functions import length

data = data.withColumn('length',length(data['text']))
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [4]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [5]:
# Feature Transformations
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

## For text columns
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

## For the label column
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [6]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [7]:
# Naive Bayes Model

from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

In [8]:
# Pipeline

from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

In [9]:
# Training and Evaluation
clean_data = clean_data.select(['label','features'])
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [10]:
train, test = clean_data.randomSplit([0.7,0.3])

In [11]:
spam_predictor = nb.fit(train)
test_results = spam_predictor.transform(test)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,13,...|[-619.80903952881...|[1.0,1.8760790417...| 0.0|
 0.0|(13424,[0,1,3,9,1...|[-573.99351687871...|[1.0,4.9369338291...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-880.14578130756...|[1.0,1.1699762469...| 0.0|
 0.0|(13424,[0,1,7,15,...|[-658.35033536708...|[1.0,5.4742005399...| 0.0|
 0.0|(13424,[0,1,14,78...|[-690.59351070344...|[1.0,1.3256639529...| 0.0|
 0.0|(13424,[0,1,23,63...|[-1305.9506932558...|[1.0,1.3665129961...| 0.0|
 0.0|(13424,[0,1,43,69...|[-615.45558566433...|[0.99995652231946...| 0.0|
 0.0|(13424,[0,1,46,17...|[-1142.6836362920...|[5.31732387582563...| 1.0|
 0.0|(13424,[0,1,416,6...|[-299.39644057754...|[1.0,4.4964723896...| 0.0|
 0.0|(13424,[0,1,498,5...|[-317.43692005547...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,1,874,1...|[-96.220870836015...|[0.99999999481136...| 0.0|
 0.0|(13424,[0,2,3,5,6...|[-2558.2635241277...|[1.0,2.8479630451...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3286.8566113028...|[1.0,1.4624923272...| 0.0|
 0.0|(13424,[0,2,4,8,1...|[-1307.5915698044...|[1.0,1.0180464752...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[-1396.7173934296...|[1.0,1.4713913307...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[-546.95679711108...|[1.0,9.3304479261...| 0.0|
 0.0|(13424,[0,2,4,11,...|[-1123.3991677610...|[1.0,3.6167662020...| 0.0|
 0.0|(13424,[0,2,7,8,3...|[-1149.7066043933...|[1.0,1.7457436746...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[-451.92328432413...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,2,7,11,...|[-1416.4366982615...|[1.0,9.4392926069...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9204299876712098